# Modelos de regresión con Keras


## Introducción

Como discutimos en los videos, a pesar de la popularidad de bibliotecas más potentes como PyTorch y TensorFlow, estas no son fáciles de usar y tienen una curva de aprendizaje pronunciada. Por ello, para las personas que apenas comienzan a aprender deep learning, no hay una biblioteca mejor para empezar que Keras.

Keras es una API de alto nivel para construir modelos de aprendizaje profundo. Ha ganado popularidad por su facilidad de uso y su simplicidad sintáctica, lo que facilita un desarrollo rápido. Como verás en este laboratorio y en otros del curso, es posible construir redes neuronales muy complejas con Keras utilizando solo unas cuantas líneas de código.

Apreciarás aún más Keras una vez que aprendas a construir modelos profundos utilizando PyTorch y TensorFlow en otros cursos.

Por lo tanto, en este laboratorio aprenderás a utilizar la biblioteca Keras para construir un modelo de regresión.

## Objetivos de este cuaderno
* Cómo utilizar la biblioteca Keras para construir un modelo de regresión
* Descargar y limpiar el conjunto de datos
* Construir una red neuronal
* Entrenar y probar la red


#### Para usar Keras, también necesitarás instalar un framework backend, como TensorFlow.

Si instalas TensorFlow versión 2.16 o superior, Keras se instalará de forma predeterminada.

Estamos utilizando la versión de TensorFlow para CPU, ya que trabajamos con conjuntos de datos pequeños.
Puedes instalar la versión de TensorFlow para GPU en tu máquina para acelerar el procesamiento de conjuntos de datos más grandes.


#### Suprimir los mensajes de advertencia de TensorFlow

Usamos el siguiente código para suprimir los mensajes de advertencia que aparecen al usar la arquitectura CPU en TensorFlow.

Es posible que desees **comentar estas líneas** si estás utilizando la arquitectura GPU.


In [2]:
import pandas as pd
import numpy as np
import keras

import warnings
warnings.simplefilter('ignore', FutureWarning)

C:\Users\Josue Santana Robled\PycharmProjects\Coursera\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


Estaremos trabajando con el mismo conjunto de datos que usamos en los videos.

El conjunto de datos trata sobre la resistencia a la compresión de diferentes muestras de concreto, basada en los volúmenes de los distintos ingredientes utilizados para fabricarlas. Los ingredientes incluyen:

* Cemento
* Escoria de alto horno
* Ceniza volante
* Agua
* Superplastificante
* Agregado grueso
* Agregado fino

## Descarga ylimpia el dataset

Vamos a descargar los datos y leerlos en un dataframe de pandas.

In [3]:
filepath='https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


La primera muestra de concreto tiene 540 metros cúbicos de cemento, 0 metros cúbicos de escoria de alto horno, 0 metros cúbicos de ceniza volante, 162 metros cúbicos de agua, 2.5 metros cúbicos de superplastificante, 1040 metros cúbicos de agregado grueso y 676 metros cúbicos de agregado fino.

Una mezcla de concreto de este tipo, con 28 días de edad, tiene una resistencia a la compresión de 79.99 MPa.

#### Verifiquemos cuántos puntos de datos tenemos


In [4]:
concrete_data.shape

(1030, 9)

Entonces, hay aproximadamente 1000 muestras para entrenar nuestro modelo. Debido a que son pocas muestras, debemos tener cuidado de no sobreajustar (overfit) los datos de entrenamiento.

Verifiquemos el conjunto de datos para detectar valores faltantes.

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

Los datos se ven muy limpios y están listos para ser utilizados en la construcción de nuestro modelo.

#### Dividir los datos en variables predictoras y variable objetivo


La variable objetivo en este problema es la resistencia de la muestra de concreto. Por lo tanto, nuestras variables predictoras serán todas las demás columnas.

In [7]:
concrete_data_columns = concrete_data.columns

In [8]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Hagamos una verificación rápida de consistencia (sanity check) de los dataframes de las variables predictoras y de la variable objetivo.

In [9]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [10]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finalmente, el último paso es normalizar los datos, restando la media y dividiendo por la desviación estándar.


In [11]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Guardemos el número de variables predictoras en n_cols, ya que necesitaremos este número al construir nuestra red.

In [12]:
n_cols = predictors_norm.shape[1] # number of predictors

## Importar paquetes de Keras

##### Vamos a importar el resto de los paquetes de la biblioteca Keras que necesitaremos para construir nuestro modelo de regresión.


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

## Construye una red Neuronal

Definamos una función que cree nuestro modelo de regresión, de manera que podamos llamarla fácilmente cada vez que queramos construir el modelo.

In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

La función anterior crea un modelo que tiene dos capas ocultas, cada una con 50 unidades ocultas.

## Entrenar y probar la red

Ahora llamemos a la función para crear nuestro modelo.

In [15]:
# build the model
model = regression_model()

A continuación, entrenaremos y probaremos el modelo al mismo tiempo utilizando el método fit. Dejaremos fuera el 30% de los datos para validación y entrenaremos el modelo durante 100 épocas.

In [16]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 1s - 32ms/step - loss: 1642.6890 - val_loss: 1143.1090
Epoch 2/100
23/23 - 0s - 4ms/step - loss: 1502.9733 - val_loss: 1012.8025
Epoch 3/100
23/23 - 0s - 4ms/step - loss: 1269.5928 - val_loss: 807.6794
Epoch 4/100
23/23 - 0s - 4ms/step - loss: 931.2803 - val_loss: 557.2693
Epoch 5/100
23/23 - 0s - 4ms/step - loss: 563.0643 - val_loss: 344.3210
Epoch 6/100
23/23 - 0s - 4ms/step - loss: 321.7903 - val_loss: 239.5842
Epoch 7/100
23/23 - 0s - 4ms/step - loss: 245.9794 - val_loss: 211.4986
Epoch 8/100
23/23 - 0s - 4ms/step - loss: 225.1988 - val_loss: 205.7051
Epoch 9/100
23/23 - 0s - 4ms/step - loss: 211.7592 - val_loss: 202.1484
Epoch 10/100
23/23 - 0s - 4ms/step - loss: 202.6850 - val_loss: 196.6662
Epoch 11/100
23/23 - 0s - 4ms/step - loss: 194.4103 - val_loss: 192.6432
Epoch 12/100
23/23 - 0s - 4ms/step - loss: 187.6250 - val_loss: 189.2095
Epoch 13/100
23/23 - 0s - 4ms/step - loss: 182.4091 - val_loss: 188.7894
Epoch 14/100
23/23 - 0s - 4ms/step - loss: 177.3407 - 

<h3>Practica Ejercicio 1</h3>

Ahora, utilizando el mismo conjunto de datos, intenta recrear un modelo de regresión que tenga cinco capas ocultas, cada una con 50 nodos y funciones de activación ReLU, una única capa de salida, y que se optimice usando el optimizador Adam.

In [17]:
def regression_model():
    input_colm = predictors_norm.shape[1] # Number of input features
    # create model
    model = Sequential()
    model.add(Input(shape=(input_colm,)))  # Set the number of input features
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))  # Output layer

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

<h3>Practica Ejercicio 2</h3>

Entrena y evalúa el modelo simultáneamente utilizando el método fit(), reservando el 10% de los datos para validación y entrenando el modelo durante 100 épocas.

In [18]:
model = regression_model()
model.fit(predictors_norm, target, validation_split=0.1, epochs=100, verbose=2)

Epoch 1/100
29/29 - 1s - 42ms/step - loss: 1477.8386 - val_loss: 890.4549
Epoch 2/100
29/29 - 0s - 4ms/step - loss: 591.4163 - val_loss: 227.5870
Epoch 3/100
29/29 - 0s - 4ms/step - loss: 251.3950 - val_loss: 216.1946
Epoch 4/100
29/29 - 0s - 4ms/step - loss: 211.5045 - val_loss: 207.8772
Epoch 5/100
29/29 - 0s - 4ms/step - loss: 195.1099 - val_loss: 201.5299
Epoch 6/100
29/29 - 0s - 4ms/step - loss: 175.2847 - val_loss: 174.8601
Epoch 7/100
29/29 - 0s - 4ms/step - loss: 160.1906 - val_loss: 173.1013
Epoch 8/100
29/29 - 0s - 4ms/step - loss: 146.1870 - val_loss: 148.9711
Epoch 9/100
29/29 - 0s - 4ms/step - loss: 134.6067 - val_loss: 150.3008
Epoch 10/100
29/29 - 0s - 4ms/step - loss: 115.2153 - val_loss: 121.7324
Epoch 11/100
29/29 - 0s - 4ms/step - loss: 100.0126 - val_loss: 127.5155
Epoch 12/100
29/29 - 0s - 3ms/step - loss: 85.5360 - val_loss: 111.8407
Epoch 13/100
29/29 - 0s - 3ms/step - loss: 72.3650 - val_loss: 88.3791
Epoch 14/100
29/29 - 0s - 3ms/step - loss: 61.6923 - val_loss

Basándonos en los resultados, observamos que:

Agregar más capas ocultas al modelo aumenta su capacidad para aprender y representar relaciones complejas dentro de los datos. Esto permite que el modelo identifique mejor los patrones, por lo que se vuelve más efectivo al ajustar los datos de entrenamiento y potencialmente mejorar sus predicciones.

Al reducir la proporción de datos reservados para validación y utilizar una mayor porción para el entrenamiento, el modelo tiene acceso a más ejemplos para aprender. Estos datos de entrenamiento adicionales ayudan al modelo a mejorar su comprensión de las tendencias subyacentes, lo que puede conducir a un mejor desempeño general.